# Exploring good ways to use graph algorithms practically
May also check out networkx, etc... to see what is simplest in python

with notes from:
http://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/ 

In [1]:
import numpy as np

In [4]:
np.zeros((12,21))

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  

In [5]:
matrix = np.zeros((12,21))

In [44]:
nodes = ['%s,%s' % (str(y), str(x))
              for y in range(0, len(matrix)) 
              for x in range(0, len(matrix[0]))]

In [45]:
nodes[0:10]

['0,0', '0,1', '0,2', '0,3', '0,4', '0,5', '0,6', '0,7', '0,8', '0,9']

Use these as names for nodes

Also parse them to index into the matrix

### Seed the data
With a connected component

And see if we can write an algorithm to find it

Steps:  
1) getting working with exact match  
2) then change to work with approx match, however defined 

(problem --> don't want to slowly get lower and lower, so need to define starting point at a class level... or discretize into bins, and only take values in same bin as starting point... and bin it relative to the that very first number)

In [10]:
matrix[0][11] = 100

In [12]:
matrix[1][11] = 0

In [13]:
matrix[2][11] = 40

In [15]:
matrix[2][12] = 40

In [16]:
matrix[2][10] = 40

In [17]:
matrix[3][10] = 40

### Build the graph
Try it out

In [148]:
class Node:
    
    def __init__(self, name, dim_y, dim_x):
        # get the name and parameters
        self.name = name
        self.y, self.x = name.split(',')
        
        # transform to int
        self.y = int(self.y)
        self.x = int(self.x)
        
        # grab the value
        self.val = matrix[self.y][self.x]
        
        # get the neighbors
        self.left = '%s,%s' % (self.y, self.x-1) if self.x > 0 else None
        self.right = '%s,%s' % (self.y, self.x+1) if self.x < dim_x-1 else None
        
        self.top = '%s,%s' % (self.y-1, self.x) if self.y > 0 else None
        self.bottom = '%s,%s' % (self.y+1, self.x) if self.y < dim_y-1 else None
        
        # list of neighbors
        self.neighbor_names = [self.left, self.right, self.top, self.bottom]
        
        # TODO: Need to create this in the BFS itself, to avoid duplicate copies of nodes
        # and also, won't have access to graph from inside node!
        self.neighbors = [graph[get_vertex_from_name(n, 21)] if n else None for n in self.neighbor_names]
        
        
    def __repr__(self):
        return 'Name: %s\n\tY: %s\n\tX: %s\n\tVAL: %s\n\tleft: %s \n\tright: %s\n\ttop: %s\n\tbottom: %s\n\n\n' % (self.name, self.y, self.x, self.val, self.left, self.right, self.top, self.bottom)
        


In [149]:
graph = [Node(n, 12, 21) for n in nodes]

In [170]:
#graph

In [48]:
graph[1].right

'1,2'

In [49]:
graph[1].left

'1,0'

In [50]:
graph[1].top

In [51]:
graph[1].bottom

'1,1'

In [52]:
graph[1].name

'0,1'

In [53]:
graph[1].val

0.0

In [54]:
len(graph)

252

In [55]:
12*21

252

In [57]:
matrix[0][11]

100.0

In [59]:
graph[11].val

100.0

In [80]:
len(matrix)

12

In [81]:
len(matrix[0])

21

# bfs

## essentially, find connected components where all values == start
then later, change so that there is some level of tolerance +/- some value from start

In [86]:
# name --> vertex
def get_vertex_from_name(name, dim_y):
    idxs = name.split(',')
    
    y = idxs[0]
    x = idxs[1]
    
    idx = int(y)*dim_y + int(x)
    
    return idx
    

In [87]:
get_vertex_from_name('1,1', 21)

22

In [131]:
graph[22]

Name: 1,1
	Y: 1
	X: 1
	VAL: 0.0
	left: 1,0 
	right: 1,2
	top: 0,1
	bottom: 2,1



In [132]:
get_vertex_from_name('11,7', 21)

238

In [133]:
graph[238]

Name: 11,7
	Y: 11
	X: 7
	VAL: 0.0
	left: 11,6 
	right: 11,8
	top: 10,7
	bottom: None



In [82]:
# original bfs
def bfs(graph, start):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited:
            visited.add(vertex)
            queue.extend(graph[vertex] - visited)
    return visited

In [171]:
# my bfs
def mybfs(graph, start, comparator):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited and vertex.val == comparator and vertex is not None:
            visited.add(vertex)
            # instantiate the neighbors list for the node here
            # attempting to do this without edges...
            
            # end creation of neighbors list
            queue.extend(set(graph[get_vertex_from_name(vertex.name, 21)].neighbors) - visited)
    return visited

In [172]:
entry = graph[get_vertex_from_name('2,11', 21)]

In [162]:
entry

Name: 2,11
	Y: 2
	X: 11
	VAL: 40.0
	left: 2,10 
	right: 2,12
	top: 1,11
	bottom: 3,11



In [163]:
entry.name

'2,11'

In [164]:
q = [entry]

In [165]:
q.pop()

Name: 2,11
	Y: 2
	X: 11
	VAL: 40.0
	left: 2,10 
	right: 2,12
	top: 1,11
	bottom: 3,11



In [166]:
vars(entry)

{'bottom': '3,11',
 'left': '2,10',
 'name': '2,11',
 'neighbor_names': ['2,10', '2,12', '1,11', '3,11'],
 'neighbors': [Name: 2,10
  	Y: 2
  	X: 10
  	VAL: 40.0
  	left: 2,9 
  	right: 2,11
  	top: 1,10
  	bottom: 3,10
  
  , Name: 2,12
  	Y: 2
  	X: 12
  	VAL: 40.0
  	left: 2,11 
  	right: 2,13
  	top: 1,12
  	bottom: 3,12
  
  , Name: 1,11
  	Y: 1
  	X: 11
  	VAL: 0.0
  	left: 1,10 
  	right: 1,12
  	top: 0,11
  	bottom: 2,11
  
  , Name: 3,11
  	Y: 3
  	X: 11
  	VAL: 0.0
  	left: 3,10 
  	right: 3,12
  	top: 2,11
  	bottom: 4,11
  
  ],
 'right': '2,12',
 'top': '1,11',
 'val': 40.0,
 'x': 11,
 'y': 2}

In [174]:
mybfs(graph, entry, 40.0)

{Name: 2,10
 	Y: 2
 	X: 10
 	VAL: 40.0
 	left: 2,9 
 	right: 2,11
 	top: 1,10
 	bottom: 3,10
 
 , Name: 2,11
 	Y: 2
 	X: 11
 	VAL: 40.0
 	left: 2,10 
 	right: 2,12
 	top: 1,11
 	bottom: 3,11
 
 , Name: 2,12
 	Y: 2
 	X: 12
 	VAL: 40.0
 	left: 2,11 
 	right: 2,13
 	top: 1,12
 	bottom: 3,12
 
 , Name: 3,10
 	Y: 3
 	X: 10
 	VAL: 40.0
 	left: 3,9 
 	right: 3,11
 	top: 2,10
 	bottom: 4,10
 
 , Name: 2,11
 	Y: 2
 	X: 11
 	VAL: 40.0
 	left: 2,10 
 	right: 2,12
 	top: 1,11
 	bottom: 3,11
 
 }

In [176]:
set( n.name for n in mybfs(graph, entry, 40.0) )

{'2,10', '2,11', '2,12', '3,10'}

### re-learned
1. Why it's important to make nodes separately from edges 
2. Make nodes first
3. Make edges afterwards, and connect those nodes
4. Encapsulate nodes and edges as separate classes
5. Put everything into a graph, after that
6. work your magic

# Networkx

In [178]:
import networkx as nx
G=nx.Graph()
G.add_node("spam")
G.add_edge(1,2)
print(list(G.nodes()))
print(list(G.edges()))


[1, 2, 'spam']
[(1, 2)]


In [179]:
G=nx.Graph()
for y in range (0, 20):
    for x in range(0, 15):
        name = '%s,%s' % (y,x)
        G.add_node(name)
        G.add_edge(y+1,x)
        G.add_edge(y-1,x)
        G.add_edge(y,x+1)
        G.add_edge(y,x-1)

# Custom Graph w/ Node and Edge Classes

In [250]:
class Node:
    
    def __init__(self, name, data):
        self.name = name
        self.data = data    
        self.edges = set()
        
    def __repr__(self):
        return "NAME: %s\nDATA:%s\nEDGES:%s\n\n\n" % (self.name, self.data, self.edges)


In [251]:
class Edge:
    
    def __init__(self, node_from, node_to):
        self.node_from = node_from
        self.node_to = node_to
    
    def __repr__(self):
        return "FROM: %s\nTO: %s\n\n" % (self.node_from, self.node_to)

In [187]:
nodes = ['%s,%s' % (str(y), str(x))
              for y in range(0, len(matrix)) 
              for x in range(0, len(matrix[0]))]

In [188]:
nodes[0:10]

['0,0', '0,1', '0,2', '0,3', '0,4', '0,5', '0,6', '0,7', '0,8', '0,9']

In [190]:
# name --> index
def get_index_from_name(name, dim_y):
    idxs = name.split(',')
    
    y = idxs[0]
    x = idxs[1]
    
    idx = int(y)*dim_y + int(x)
    
    return idx

In [202]:
get_index_from_name('1,9', 21)

30

In [205]:
def get_matrix_value_from_name(matrix, name):
    y, x = name.split(',')
    val = matrix[int(y)][int(x)]
    return val

In [206]:
get_matrix_value_from_name(matrix, '1,9')

0.0

In [207]:
get_matrix_value_from_name(matrix, '0,11')

100.0

In [208]:
get_matrix_value_from_name(matrix, '1,11')

0.0

In [209]:
get_matrix_value_from_name(matrix, '2,11')

40.0

In [324]:
class Graph:
    """ Graph for a 2D matrix """
    
    def __init__(self, matrix):
        # keep the underlying matrix representation
        self.matrix = matrix
        
        # get matrix dimensions
        self.x_dim = len(matrix)
        self.y_dim = len(matrix[0])
        
        # get node names
        self.node_names = ['%s,%s' % (str(y), str(x))  # reverse x & y for names
                              for y in range(0, self.x_dim) 
                              for x in range(0, self.y_dim)]
        
        # instantiate all the nodes
        self.nodes = [Node(n, self.get_matrix_value_from_name(self.matrix, n)) 
                      for n in self.node_names]
        
        # connect all the edges        
        for n in self.nodes:
            n.edges = self.get_edges(n.name) 
        
        
    def get_edges(self, name):
        # get the parameters
        y, x = name.split(',')
        
        # transform to int
        y = int(y)
        x = int(x)
        
        # get the neighbors --> compare x_dim vs y_dim because python flips the indices for 2D graphs
        # instantiate x_first, but index y_first
        to_left = '%s,%s' % (y, x-1) if x > 0 else None
        to_right = '%s,%s' % (y, x+1) if x < self.y_dim-1 else None 
        
        to_top = '%s,%s' % (y-1, x) if y > 0 else None
        to_bottom = '%s,%s' % (y+1, x) if y < self.x_dim-1 else None
        
        to_names = [to_left, to_right, to_top, to_bottom]
        
        edges = set(Edge(name, to) if to else None 
                    for to in to_names)
    
        if None in edges:
            edges.remove(None)
        
        return edges
 
    def get_matrix_value_from_name(self, matrix, name):
        y, x = name.split(',')
        val = matrix[int(y)][int(x)]
        return val

    # name --> index
    def get_index_from_name(self, name):
        idxs = name.split(',')

        y = idxs[0]
        x = idxs[1]

        idx = int(y)*self.y_dim + int(x)

        return idx
    
    def get_node(self, name):
        self.nodes[self.get_index_from_name(name)]

In [325]:
g = Graph(matrix)

In [326]:
#g.node_names

In [327]:
g.get_index_from_name('1,11')

32

In [328]:
g.nodes[32]

NAME: 1,11
DATA:0.0
EDGES:set([FROM: 1,11
TO: 1,12

, FROM: 1,11
TO: 2,11

, FROM: 1,11
TO: 1,10

, FROM: 1,11
TO: 0,11

])



In [329]:
g.nodes

[NAME: 0,0
 DATA:0.0
 EDGES:set([FROM: 0,0
 TO: 0,1
 
 , FROM: 0,0
 TO: 1,0
 
 ])
 
 , NAME: 0,1
 DATA:0.0
 EDGES:set([FROM: 0,1
 TO: 0,0
 
 , FROM: 0,1
 TO: 1,1
 
 , FROM: 0,1
 TO: 0,2
 
 ])
 
 , NAME: 0,2
 DATA:0.0
 EDGES:set([FROM: 0,2
 TO: 0,3
 
 , FROM: 0,2
 TO: 1,2
 
 , FROM: 0,2
 TO: 0,1
 
 ])
 
 , NAME: 0,3
 DATA:0.0
 EDGES:set([FROM: 0,3
 TO: 0,4
 
 , FROM: 0,3
 TO: 0,2
 
 , FROM: 0,3
 TO: 1,3
 
 ])
 
 , NAME: 0,4
 DATA:0.0
 EDGES:set([FROM: 0,4
 TO: 0,3
 
 , FROM: 0,4
 TO: 1,4
 
 , FROM: 0,4
 TO: 0,5
 
 ])
 
 , NAME: 0,5
 DATA:0.0
 EDGES:set([FROM: 0,5
 TO: 0,6
 
 , FROM: 0,5
 TO: 1,5
 
 , FROM: 0,5
 TO: 0,4
 
 ])
 
 , NAME: 0,6
 DATA:0.0
 EDGES:set([FROM: 0,6
 TO: 0,7
 
 , FROM: 0,6
 TO: 0,5
 
 , FROM: 0,6
 TO: 1,6
 
 ])
 
 , NAME: 0,7
 DATA:0.0
 EDGES:set([FROM: 0,7
 TO: 0,6
 
 , FROM: 0,7
 TO: 1,7
 
 , FROM: 0,7
 TO: 0,8
 
 ])
 
 , NAME: 0,8
 DATA:0.0
 EDGES:set([FROM: 0,8
 TO: 1,8
 
 , FROM: 0,8
 TO: 0,9
 
 , FROM: 0,8
 TO: 0,7
 
 ])
 
 , NAME: 0,9
 DATA:0.0
 EDGES:set([F

In [312]:
# get entry point
entry = g.nodes[g.get_index_from_name('2,11')]

In [313]:
entry

NAME: 2,11
DATA:40.0
EDGES:set([FROM: 2,11
TO: 2,12

, FROM: 2,11
TO: 3,11

, FROM: 2,11
TO: 2,10

, FROM: 2,11
TO: 1,11

])



In [349]:
def mybfs(graph, start, comparator):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited and vertex.data == comparator and vertex is not None:
            visited.add(vertex)
            
            # grab edges and the attached nodes
            edges = graph.nodes[graph.get_index_from_name(vertex.name)].edges
            nodes = set(graph.nodes[graph.get_index_from_name(e.node_to)] for e in edges)
            
            # extend, ignoring visited nodes so far
            queue.extend(nodes - visited)
    return visited

In [350]:
g.nodes[g.get_index_from_name('1,11')]

NAME: 1,11
DATA:0.0
EDGES:set([FROM: 1,11
TO: 1,12

, FROM: 1,11
TO: 2,11

, FROM: 1,11
TO: 1,10

, FROM: 1,11
TO: 0,11

])



In [351]:
g.get_node('1,11')

In [352]:
mybfs(g, entry, 40.0)

{NAME: 2,11
 DATA:40.0
 EDGES:set([FROM: 2,11
 TO: 2,12
 
 , FROM: 2,11
 TO: 3,11
 
 , FROM: 2,11
 TO: 2,10
 
 , FROM: 2,11
 TO: 1,11
 
 ])
 
 , NAME: 2,10
 DATA:40.0
 EDGES:set([FROM: 2,10
 TO: 2,11
 
 , FROM: 2,10
 TO: 3,10
 
 , FROM: 2,10
 TO: 2,9
 
 , FROM: 2,10
 TO: 1,10
 
 ])
 
 , NAME: 2,11
 DATA:40.0
 EDGES:set([FROM: 2,11
 TO: 2,10
 
 , FROM: 2,11
 TO: 1,11
 
 , FROM: 2,11
 TO: 2,12
 
 , FROM: 2,11
 TO: 3,11
 
 ])
 
 , NAME: 2,12
 DATA:40.0
 EDGES:set([FROM: 2,12
 TO: 3,12
 
 , FROM: 2,12
 TO: 2,11
 
 , FROM: 2,12
 TO: 1,12
 
 , FROM: 2,12
 TO: 2,13
 
 ])
 
 , NAME: 3,10
 DATA:40.0
 EDGES:set([FROM: 3,10
 TO: 3,11
 
 , FROM: 3,10
 TO: 3,9
 
 , FROM: 3,10
 TO: 2,10
 
 , FROM: 3,10
 TO: 4,10
 
 ])
 
 }

In [353]:
ns = mybfs(g, entry, 40.0)

In [358]:
set(list(n.name for n in ns))

{'2,10', '2,11', '2,12', '3,10'}

# Everything Together


In [ ]:
########
# NODE #
########
class Node:
    
    def __init__(self, name, data):
        self.name = name
        self.data = data    
        self.edges = set()
        
    def __repr__(self):
        return "NAME: %s\nDATA:%s\nEDGES:%s\n\n\n" % (self.name, self.data, self.edges)

########
# EDGE #
########
class Edge:
    
    def __init__(self, node_from, node_to):
        self.node_from = node_from
        self.node_to = node_to
    
    def __repr__(self):
        return "FROM: %s\nTO: %s\n\n" % (self.node_from, self.node_to)

#########
# GRAPH #
#########
class Graph:
    """ Graph for a 2D matrix """
    
    def __init__(self, matrix):
        # keep the underlying matrix representation
        self.matrix = matrix
        
        # get matrix dimensions
        self.x_dim = len(matrix)
        self.y_dim = len(matrix[0])
        
        # get node names
        self.node_names = ['%s,%s' % (str(y), str(x))  # reverse x & y for names
                              for y in range(0, self.x_dim) 
                              for x in range(0, self.y_dim)]
        
        # instantiate all the nodes
        self.nodes = [Node(n, self.get_matrix_value_from_name(self.matrix, n)) 
                      for n in self.node_names]
        
        # connect all the edges        
        for n in self.nodes:
            n.edges = self.get_edges(n.name) 
        
        
    def get_edges(self, name):
        # get the parameters
        y, x = name.split(',')
        
        # transform to int
        y = int(y)
        x = int(x)
        
        # get the neighbors --> compare x_dim vs y_dim because python flips the indices for 2D graphs
        # instantiate x_first, but index y_first
        to_left = '%s,%s' % (y, x-1) if x > 0 else None
        to_right = '%s,%s' % (y, x+1) if x < self.y_dim-1 else None 
        
        to_top = '%s,%s' % (y-1, x) if y > 0 else None
        to_bottom = '%s,%s' % (y+1, x) if y < self.x_dim-1 else None
        
        to_names = [to_left, to_right, to_top, to_bottom]
        
        edges = set(Edge(name, to) if to else None 
                    for to in to_names)
    
        if None in edges:
            edges.remove(None)
        
        return edges
 
    def get_matrix_value_from_name(self, matrix, name):
        y, x = name.split(',')
        val = matrix[int(y)][int(x)]
        return val

    # name --> index
    def get_index_from_name(self, name):
        idxs = name.split(',')

        y = idxs[0]
        x = idxs[1]

        idx = int(y)*self.y_dim + int(x)

        return idx
    
    def get_node(self, name):
        self.nodes[self.get_index_from_name(name)]
        

#######
# BFS #
#######
def mybfs(graph, start, comparator):
    visited, queue = set(), [start]
    while queue:
        vertex = queue.pop(0)
        if vertex not in visited and vertex.data == comparator and vertex is not None:
            visited.add(vertex)
            
            # grab edges and the attached nodes
            edges = graph.nodes[graph.get_index_from_name(vertex.name)].edges
            nodes = set(graph.nodes[graph.get_index_from_name(e.node_to)] for e in edges)
            
            # extend, ignoring visited nodes so far
            queue.extend(nodes - visited)
    return visited